In [1]:
import numpy as np
var = 0.1
d = 5
N = 100000
K = 30
a = np.random.uniform(size = d)
b = np.random.uniform(size = (d, d))
b = b @ np.transpose(b)
inv_cov = np.linalg.inv(b)
z0 = np.power(np.linalg.det(2 * np.pi * b), -0.5 * (1/K))
zf = np.power(np.linalg.det(2 * np.pi * b), -0.5)
z = 1
def get_prob(proposal, X, cov):
  A = - 0.5 * (np.sum((proposal - a) @ cov * (proposal - a), axis = 1))
  D = - 0.5 * (np.sum((X - a) @ cov * (X - a), axis = 1))
  return np.exp(A - D)

def get_weights(X, next_cov, curr_cov):
  A = - 0.5 * (np.sum((X - a) @ next_cov * (X - a), axis = 1))
  D = - 0.5 * (np.sum((X - a) @ curr_cov * (X - a), axis = 1))
  return np.exp(A - D)

# Samples initialization
X = np.random.multivariate_normal(a, (K/1) * b, size = N)
W = np.full(N, 1/N)

for i in range(2, K + 1):
  # Resample
  I = np.random.choice(np.arange(N), size=N, p=W)
  X = X[I, :]
  W = np.full(N, 1/N)

  # MCMC step
  for j in range(10):
    R = np.random.multivariate_normal(np.zeros(d), np.identity(d), size = N)
    proposal = X + var * R
    U = np.random.uniform(size = N)
    alpha = get_prob(proposal, X, (i/K) * inv_cov)
    I = U < alpha
    X[I] = proposal[I]

  # Computing weights
  W = get_weights(X, ((i + 1)/K) * inv_cov, (i/K) * inv_cov)
  z *= np.sum(W)/N
  z0 = np.power(np.linalg.det(2 * np.pi * (K/(i-1)) * b), -0.5)
  zf = np.power(np.linalg.det(2 * np.pi * (K/i) * b), -0.5)
  W = W/np.sum(W)
  
sample_mean = W.T @ X
sample_cov = np.cov(np.transpose(X))
print("True Mean: ", a[:5])
print("Sample Mean: ", sample_mean[:5])
print("True Covariance: ", b[:5, :5])
print("Sample Covariance: ", sample_cov[:5, :5])
print("est ratio ", z)

True Mean:  [0.84030261 0.47553728 0.83215873 0.13266596 0.69726655]
Sample Mean:  [0.84425793 0.47574291 0.85233046 0.14509961 0.72811593]
True Covariance:  [[1.41949603 1.27172123 0.9833405  1.64093079 0.99288176]
 [1.27172123 1.38584626 0.91145774 1.46165923 0.784073  ]
 [0.9833405  0.91145774 1.28499172 1.33490533 1.27527137]
 [1.64093079 1.46165923 1.33490533 2.03866962 1.42801905]
 [0.99288176 0.784073   1.27527137 1.42801905 1.70453252]]
Sample Covariance:  [[1.41470909 1.26642862 0.97755017 1.63551096 0.98866449]
 [1.26642862 1.38108729 0.906116   1.45552803 0.77745903]
 [0.97755017 0.906116   1.28543871 1.33113916 1.28021701]
 [1.63551096 1.45552803 1.33113916 2.03395967 1.42800369]
 [0.98866449 0.77745903 1.28021701 1.42800369 1.71844303]]
est ratio  0.00026715775171946333
